<a href="https://colab.research.google.com/github/EricDeop/NLP-Employee-Survey-Project/blob/NLP---Employee-Survey-Project/MISC_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

import numpy as np
import pandas as pd

path = "/content/drive/MyDrive/LeaderComment.csv"

df = pd.read_csv(path)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,SURVEYRESPONDERID,AVERAGE_RESPONSE_DATE,YEAR,MONTH,GENDER,SURVEYQUESTIONESSENCEID,COMMENT
0,17296164,1/7/21,2021,1,Male,30013,Melvins professional demeanor and interpersona...
1,17311889,1/7/21,2021,1,Female,30014,Proved better supervision and support.
2,17334983,1/7/21,2021,1,Male,30013,Proven experience
3,17340598,1/2/21,2021,1,Male,30014,"Let the team do what they were hired to do, so..."
4,17340874,1/2/21,2021,1,Male,30013,They consistently look for opportunities to ma...


# MISCELLANEOUS EXPLORATORY DATA ANALYSIS

In [2]:
#Importing necessary packages
import spacy
import re
import string
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df['GENDER'] = df['GENDER'].str.lower()

df = df.replace({'GENDER' : {'do not wish to disclose' : 'prefer not to answer',
                        'cisgender' : 'gender not listed', 'm' : 'male', 
                        'f' : 'female'}})

df = df[df["GENDER"].str.contains(
    "non-binary|other gender|gender not listed|gender identity not listed|other gender"
    ) == False]

print("\n")

df['GENDER'].value_counts()

df['GENDER'] = df['GENDER'].str.lower()

df = df.replace({'GENDER' : {'do not wish to disclose' : 'prefer not to answer',
                        'cisgender' : 'gender not listed', 'm' : 'male', 
                        'f' : 'female'}})

df = df[df["GENDER"].str.contains(
    "non-binary|other gender|gender not listed|gender identity not listed|other gender"
    ) == False]

print("\n")

df['GENDER'].value_counts()

# Removing Nan Values

print(df.isna().sum().sort_values(ascending = False))

#NaN values only found in comment column
#Removal of those 4 comments
df = df.dropna()

df.shape

# Dropping AVERAGE_RESPONSE_DATE, YEAR, and MONTH columns as they will 
# not be relevant to this analysis

df = df.drop(columns=['AVERAGE_RESPONSE_DATE', 'YEAR', 'MONTH'], axis=1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app




COMMENT                    4
SURVEYRESPONDERID          0
AVERAGE_RESPONSE_DATE      0
YEAR                       0
MONTH                      0
GENDER                     0
SURVEYQUESTIONESSENCEID    0
dtype: int64


,SURVEYRESPONDERID,GENDER,SURVEYQUESTIONESSENCEID,COMMENT
0,17296164,male,30013,Melvins professional demeanor and interpersona...
1,17311889,female,30014,Proved better supervision and support.
2,17334983,male,30013,Proven experience
3,17340598,male,30014,"Let the team do what they were hired to do, so..."
4,17340874,male,30013,They consistently look for opportunities to ma...


In [4]:
female = df[df["GENDER"].str.match("female") == True]
female_30014 = female[female["SURVEYQUESTIONESSENCEID"] == 30014]

male = df[df["GENDER"].str.match("male") == True]
male_30014 = male[male["SURVEYQUESTIONESSENCEID"] == 30014]

pnta = df[df["GENDER"].str.match("prefer not to answer") == True]
pnta_30014 = pnta[pnta["SURVEYQUESTIONESSENCEID"] == 30014]

In [5]:
female_comments = female_30014[female_30014["COMMENT"].str.contains("build") == True]

male_comments = male_30014[male_30014["COMMENT"].str.contains("transparent|honest|direct") == True]

pnta_comments = pnta_30014[pnta_30014['COMMENT'].str.contains("politic|senior|accountable|learn") == True]

In [6]:
pd.set_option('display.max_colwidth', None)
import random

random_comment_female = female_comments['COMMENT']

random_comment_male = male_comments['COMMENT']

random_comment_pnta = pnta_comments['COMMENT']

sampling_female = random.sample(list(random_comment_female), k=4) 
sampling_male = random.sample(list(random_comment_male), k=3) 
sampling_pnta = random.sample(list(random_comment_pnta), k=3) 


In [7]:
print(sampling_female)

['This survey is confusing as to whom we are evaluating. It has several questions regarding Monarch Management team whom I do not necessarily work with directly for anything so it is difficulty to evaluate.  I feel that my direct manager in MY building is very supportive and good to work with. ', 'I think he could see what happens at building levels and what it is like working in the buildings when decisions are made at Central Office. I feel that where is a disconnect. ', 'Values (including who and what you value) show up in your actions and your words. Here\'s a recap: \r\n\r\nWe\'re doing great financially per the monthly meetings all of 2020. Then: we\'ll have to see how it goes for paying out a return on your time investment during a pay cut. In the meantime, we bought a new building. Then "more revenue, more revenue, more revenue." Then, I believe the firm has always been more about people than money. \r\n\r\nThese statements and actions demonstrate lack of clarity in direction. 

In [8]:
print(sampling_male)

['Listen to our customers and respect the customers. Forcing consumer software to do updates or telemetry does not help. (Bloatware, "Vendor Malware") Also, adding value to the applications we pre-install and make them optional. Again respect the customers. For commercial software, understand the use cases and expect Dell Images to get wiped. Provide more value in commercial solutions. \nWork on modernizing our applications to be a hybrid Service/cloud that adds real value to the end-user.\nOn my personal laptop, I un-install all the Dell applications and just leave Dell Command Update, to do updates. (We have like 4 ways we can update the machine currently pre-installed, and updates don\'t match) Normally I just go directly to dell.com to do my updates since Dell Command Update has a 2-week delay in the catalog or has issues.\nAlso, more realistic training on Security with live instructors and in-person if possible. Today I read this article https://www.dell.com/support/kbdoc/en-us/00

In [9]:
print(sampling_pnta)

['At this point, I do not know. I have had concerns and issues over my time here that I have brought to management, compliance officers, and skip-level bosses and had no follow up from anyone other than the senior retreat manager, Annette. ', "Stay in your lane and stop spending so much time trying to be PC and focus on making Redfin a good place to work for everyone regardless of our race, gender, age, etc. Bring us together as fellow teammates and stop emphasizing our differences.\nPlease refrain from giving advice on how we should identify with our pronouns in email signature line. He was late to the game on that and it just reeked of pandering and came across quite cringe worthy. More importantly, this is a complete overstep into our personal lives and he has absolutely no place or invitation to tell us how we should identify ourselves. He's a CEO not our personal life coach or priest - please stay out of politics and our personal choices.", "Push assembly depts. to improve current

In [10]:
print(len(random_comment_female))
print(len(random_comment_male))
print(len(random_comment_pnta))

580
2818
1001


In [11]:
# Converting all text to lowercase and removing punctuations

def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]',' ', text)
  text = re.sub('\[.*?\#]', '', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  return text

cleaning = lambda x: clean_text(x)

df_clean = pd.DataFrame(df['COMMENT'].apply(cleaning))

df["COMMENT"] = df_clean

print(df['COMMENT'].head(10),"\n")
print(df['COMMENT'].tail(10))

0                                                                                                                                                                                                                      melvins professional demeanor and interpersonal skills inspire confidence 
1                                                                                                                                                                                                                                                          proved better supervision and support 
2                                                                                                                                                                                                                                                                              proven experience 
3                                                                                                                                 

In [12]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

nlp = spacy.load('en_core_web_lg', disable = ['parser','ner'])

print(nlp.Defaults.stop_words)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
{'whenever', 'across', 'when', 'unless', 'move', 'to', 'him', 'twelve', 'well', "'m", 'through', 'now', 'too', 'else', 'quite', 'say', 'anywhere', 'beforehand', 'the', 'everything', 'what', 'former', 'ourselves', 'while', 'her', 'few', 'although', 'whether', 'he', 'must', 'mostly', 'were', 'hereafter', 'rather', 'keep', 'might', 'them', 'due', 'nor', '‘d', 'throughout', 'after', 'there', 'does', 'eleven', 'his', 'more', 'noone', 'yours', 'side', 'whole', 'below', 'within', 'though', 'if', 'seeming', 'your', 'than', 'has', 'eight', 'hence', 'already', '’ve', '’ll', 'somewhere', 'twenty', 'something', 'almost', 'everywhere', 'sometimes', 'out', 'namely', 'thru', 'besides', "'ll", 'these', 'seems', 'one', 'seemed', 'until', 'up', 'wherever', 'never', 'toward', 'someone', 'perhaps', 'top', 'their', 'off', 'about', 'which', 'from', 'for', 'show', 'is', 'are', 'those', 'become', 'our', 'herse

In [13]:
df_inspire = df[df['SURVEYQUESTIONESSENCEID'] == 30013]
df_gain = df[df['SURVEYQUESTIONESSENCEID'] == 30014]

print(df_inspire.head(3))
df_gain.head(3)

   SURVEYRESPONDERID GENDER  SURVEYQUESTIONESSENCEID  \
0           17296164   male                    30013   
2           17334983   male                    30013   
4           17340874   male                    30013   

                                                                                                                               COMMENT  
0                                                           melvins professional demeanor and interpersonal skills inspire confidence   
2                                                                                                                   proven experience   
4  they consistently look for opportunities to market the es brand and look for ways to improve the work environment of the employees   


,SURVEYRESPONDERID,GENDER,SURVEYQUESTIONESSENCEID,COMMENT
1,17311889,female,30014,proved better supervision and support
3,17340598,male,30014,let the team do what they were hired to do sometimes it feels like the leaders have a fomo or an i have to do it for it to be done right mentality
8,17341752,female,30014,quit pitting the different shifts and departments against each other treat all fairly quit playing favorites make sure the distribution of work is equal all around make sure everyone is pulling their weight if a task is not suited to ones personality try something different


In [14]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in nlp.Defaults.stop_words] for doc in texts]

In [15]:
inspire_words_nostops = remove_stopwords(df_inspire['COMMENT'])

gain_words_nostops = remove_stopwords(df_gain['COMMENT'])

In [16]:
import nltk
nltk.download('punkt')

long_list_inspire = [item for sublist in inspire_words_nostops for item in sublist]  
long_list_gain = [item for sublist in gain_words_nostops for item in sublist] 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
bigrams_inspire = nltk.bigrams(long_list_inspire)
bigrams_gain = nltk.bigrams(long_list_gain)

In [18]:
frequency_inspire = nltk.FreqDist(bigrams_inspire)
frequency_gain = nltk.FreqDist(bigrams_gain)

In [20]:
frequency_inspire.most_common(50)

[(('feel', 'like'), 1998),
 (('cares', 'employees'), 1452),
 (('makes', 'feel'), 1162),
 (('genuinely', 'cares'), 1075),
 (('care', 'employees'), 1064),
 (('inspires', 'confidence'), 1042),
 (('right', 'direction'), 874),
 (('truly', 'cares'), 845),
 (('forward', 'thinking'), 832),
 (('new', 'ideas'), 824),
 (('team', 'members'), 795),
 (('leadership', 'team'), 772),
 (('best', 'interest'), 743),
 (('hard', 'work'), 741),
 (('positive', 'attitude'), 720),
 (('takes', 'time'), 687),
 (('great', 'job'), 650),
 (('open', 'door'), 646),
 (('genuinely', 'care'), 620),
 (('company', 'employees'), 609),
 (('don', 'know'), 609),
 (('cares', 'people'), 603),
 (('long', 'term'), 559),
 (('open', 'communication'), 540),
 (('willing', 'listen'), 540),
 (('good', 'job'), 532),
 (('door', 'policy'), 520),
 (('great', 'leader'), 519),
 (('track', 'record'), 515),
 (('decision', 'making'), 507),
 (('day', 'day'), 491),
 (('direction', 'company'), 491),
 (('hard', 'working'), 485),
 (('open', 'honest')

In [21]:
frequency_gain.most_common(50)

[(('feel', 'like'), 1841),
 (('don', 'know'), 1732),
 (('team', 'members'), 1123),
 (('day', 'day'), 1051),
 (('upper', 'management'), 869),
 (('gain', 'confidence'), 792),
 (('work', 'home'), 708),
 (('feels', 'like'), 703),
 (('long', 'term'), 688),
 (('listen', 'employees'), 600),
 (('decision', 'making'), 548),
 (('better', 'communication'), 546),
 (('work', 'life'), 536),
 (('hard', 'work'), 521),
 (('senior', 'leadership'), 495),
 (('spend', 'time'), 487),
 (('life', 'balance'), 486),
 (('people', 'work'), 475),
 (('don', 'think'), 475),
 (('listen', 'concerns'), 468),
 (('don', 'feel'), 457),
 (('better', 'pay'), 453),
 (('leadership', 'team'), 446),
 (('increase', 'pay'), 440),
 (('care', 'employees'), 421),
 (('cost', 'living'), 419),
 (('working', 'home'), 395),
 (('good', 'job'), 395),
 (('actually', 'listen'), 372),
 (('employees', 'feel'), 361),
 (('executive', 'team'), 354),
 (('employees', 'work'), 344),
 (('senior', 'management'), 332),
 (('making', 'decisions'), 331),


In [22]:
#Adding my own stop words to list to get a better topic model

nlp.Defaults.stop_words |= {'employees', 'company', 'work', 'people', 'like', 
                            'feel', 'leadership','know', 'time', 'staff', 
                            'better', 'leader', 'good', 'team', 'don','confidence',
                            'great', 'ceo', 'care', 'new', 'job', 'business',
                            'things', 'listen', 'decisions','doesn','u'}

In [23]:
print(inspire_words_nostops[0:2])

[['melvins', 'professional', 'demeanor', 'interpersonal', 'skills', 'inspire', 'confidence'], ['proven', 'experience']]


In [24]:
def lemmatization(texts, allowed_postags=['NOUN','ADJ', 'VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

inspire_final_words = lemmatization(inspire_words_nostops, allowed_postags = [
    'NOUN','ADJ', 'VERB','ADV'])

gain_final_words = lemmatization(gain_words_nostops, allowed_postags = [
    'NOUN','ADJ', 'VERB','ADV'])

print(inspire_final_words[:1])
print(gain_final_words[:1])

[['melvin', 'professional', 'demeanor', 'interpersonal', 'skill', 'inspire', 'confidence']]
[['prove', 'well', 'supervision', 'support']]


In [25]:
long_list_inspire = [item for sublist in inspire_final_words for item in sublist]   

long_list_gain = [item for sublist in gain_final_words for item in sublist]  

In [26]:
from collections import Counter

inspire_word_freq = Counter(long_list_inspire)
gain_word_freq = Counter(long_list_gain)

In [27]:
inspire_word_freq.most_common(20)

[('company', 17605),
 ('employee', 15174),
 ('work', 13398),
 ('care', 12385),
 ('good', 10814),
 ('leader', 10169),
 ('feel', 8830),
 ('know', 8031),
 ('leadership', 6689),
 ('make', 6379),
 ('team', 6285),
 ('people', 6229),
 ('confidence', 6096),
 ('great', 5953),
 ('time', 5336),
 ('open', 5232),
 ('inspire', 5069),
 ('business', 4994),
 ('help', 4843),
 ('positive', 4668)]

In [28]:
gain_word_freq.most_common(20)

[('employee', 18376),
 ('work', 17731),
 ('company', 11814),
 ('people', 11239),
 ('need', 9492),
 ('time', 8752),
 ('feel', 8414),
 ('know', 7867),
 ('pay', 7825),
 ('well', 7617),
 ('team', 7499),
 ('staff', 7488),
 ('change', 6674),
 ('job', 6008),
 ('listen', 5961),
 ('leader', 5793),
 ('thing', 5297),
 ('decision', 5293),
 ('leadership', 5217),
 ('good', 5142)]

In [29]:
long_string_inspirecounter = ' '.join(inspire_word_freq)

long_string_gaincounter = ' '.join(gain_word_freq)

doc_inspire_counter = nlp(long_string_inspirecounter)

doc_gain_counter = nlp(long_string_gaincounter)

In [32]:
from spacy.matcher import Matcher 
matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'VERB'}, {'POS':'ADJ'}] 
matcher.add('ADJ_PHRASE', [pattern]) 
matches_inspire = matcher(doc_inspire_counter, as_spans=True) 
phrases_inspire = [] 
for span in matches_inspire:
    phrases_inspire.append(span.text.lower())
    phrase_inspire_freq = Counter(phrases_inspire)
phrase_inspire_freq.most_common(30)

[('terminate bad', 1),
 ('interact different', 1),
 ('make sure', 1),
 ('expect willing', 1),
 ('want focused', 1),
 ('learn available', 1),
 ('build good', 1),
 ('grow great', 1),
 ('profit genuine', 1),
 ('communicate darkest', 1),
 ('ve long', 1),
 ('come small', 1),
 ('present positive', 1),
 ('bring incredible', 1),
 ('doesn popular', 1),
 ('reassure tough', 1),
 ('prepare intelligent', 1),
 ('consider valuable', 1),
 ('meeting favorite', 1),
 ('see upset', 1),
 ('motivate appreciative', 1),
 ('remind compliment', 1),
 ('lose sparkle', 1),
 ('belittle embarrass', 1),
 ('play active', 1),
 ('vary competent', 1),
 ('treat helpful', 1),
 ('hire welcome', 1),
 ('grasp necessary', 1),
 ('stay relevant', 1)]

In [31]:
matches_gain = matcher(doc_gain_counter, as_spans=True) 
phrases_gain = [] 
for span in matches_gain:
    phrases_gain.append(span.text.lower())
    phrase_gain_freq = Counter(phrases_gain)
phrase_gain_freq.most_common(30)

[('prove well', 1),
 ('playing favorite', 1),
 ('penalize bad', 1),
 ('stick great', 1),
 ('review familiar', 1),
 ('office necessary', 1),
 ('speak loud', 1),
 ('send major', 1),
 ('implement big', 1),
 ('building clean', 1),
 ('create novel', 1),
 ('prop old', 1),
 ('attend simple', 1),
 ('walk endure', 1),
 ('hold accountable', 1),
 ('involve new', 1),
 ('explain minute', 1),
 ('resign public', 1),
 ('acknowledge real', 1),
 ('micromanage overhear', 1),
 ('see biased', 1),
 ('reflect knowledgeable', 1),
 ('excuse double', 1),
 ('get confident', 1),
 ('provide yearly', 1),
 ('break suppose', 1),
 ('offer executive', 1),
 ('obligated ample', 1),
 ('retain worth', 1),
 ('enable frustrated', 1)]